In [1]:
import os.path
import subprocess

import pandas as pd

from kcmc_instance import KCMC_Instance

from gurobi_models import gurobi_multi_flow, gurobi_single_flow, get_installation


from tqdm.notebook import tqdm
    


# list_serialized_instances = pd.read_csv('/data/instances.csv', header=None)[0].tolist()[1000:1010]
list_serialized_instances = open('/data/instances_gupta.txt', 'r').read().splitlines()[:1]

K = 3
M = 2

GUROBI_TIMEOUT = 60
GUROBI_THREADS = 4

In [2]:
def get_cache(prefix):
    file = f'/data/{prefix}_{K}_{M}.pq'
    if os.path.exists(file):
        data = pd.read_parquet(file)
        cache = set(data['serialized_instance'].unique())
        return data.to_dict(orient='records'), cache
    return [], set()

def to_cache(prefix, df):
    file = f'/data/{prefix}_{K}_{M}.pq'
    df.to_parquet(file)

In [3]:
gurobi_mf_data, cache = get_cache('gurobi_mf')
for ser_instance in tqdm(list_serialized_instances):
    if ser_instance in cache: continue
    try:
        obj_instance = KCMC_Instance(ser_instance, False, True, True)
    except: continue
        
    model, X, Y = gurobi_multi_flow(K, M, obj_instance, GUROBI_TIMEOUT, GUROBI_THREADS)
    results = model.optimize()
    
    gurobi_mf_data.append({
        'serialized_instance': ser_instance,
        'gurobi_mf_status': results['status'],
        'gurobi_mf_gap': results['mip_gap'],
        'gurobi_mf_feasible': results['status'] == 'OPTIMAL',
        'gurobi_mf_runtime': results['gurobi_runtime']
    })
    
    #print('\n\n', obj_instance)
    #print(results['status'], results['gurobi_runtime'])
    if results['status'] in {'OPTIMAL'}:
        wsn, installation = get_installation(X)
        #print(len(wsn), len(installation), round(len(installation)/len(wsn), 3))        
        gurobi_mf_data[-1].update({'gurobi_mf_installation': installation,
                                   'gurobi_mf_size': len(installation),
                                   'gurobi_mf_quality': len(installation)/len(wsn)})
        
gurobi_mf_data = pd.DataFrame(gurobi_mf_data)
to_cache('gurobi_mf', gurobi_mf_data)
len(gurobi_mf_data)

  0%|          | 0/1 [00:00<?, ?it/s]

[client] info  : Using libcurl/7.76.1 OpenSSL/1.1.1k
[client] info  : Default certificate path is /etc/ssl/certs/ca-certificates.crt
[client] info  : No HTTPS proxy
[client] info  : No HTTP proxy
[client] info  : Validate web license token 813d29df-e6ac-42e9-bff6-2178b73d0b3f for container d948a34dccd1eeb65e36687706e634ec1f7776fa67e1ab70215ca0163d01bee1
Academic license - for non-commercial use only - registered to josefrf@ufmg.br
[client] info  : Web license token valid
Using license file /opt/gurobi/gurobi.lic
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID
Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
[client] info  : Validate web license token 813d29df-e6ac-42e9-bff6-2178b73d0b3f for container d948a34dccd1eeb65e36687706e634ec1f7776fa67e1ab70215ca0163d01bee1
Aca

1

In [4]:
gurobi_sf_data, cache = get_cache('gurobi_sf')
for ser_instance in tqdm(list_serialized_instances):
    if ser_instance in cache: continue
    try:
        obj_instance = KCMC_Instance(ser_instance, False, True, True)
    except: continue
        
    model, X, Y = gurobi_single_flow(K, M, obj_instance, GUROBI_TIMEOUT, GUROBI_THREADS)
    results = model.optimize()
    
    gurobi_sf_data.append({
        'serialized_instance': ser_instance,
        'gurobi_sf_gap': results['mip_gap'],
        'gurobi_sf_status': results['status'],
        'gurobi_sf_feasible': results['status'] == 'OPTIMAL',
        'gurobi_sf_runtime': results['gurobi_runtime']
    })
    
    #print('\n\n', obj_instance)
    #print(results['status'], results['gurobi_runtime'])
    if results['status'] in {'OPTIMAL'}:
        wsn, installation = get_installation(X)
        #print(len(wsn), len(installation), round(len(installation)/len(wsn), 3))        
        gurobi_sf_data[-1].update({'gurobi_sf_installation': installation,
                                   'gurobi_sf_size': len(installation),
                                   'gurobi_sf_quality': len(installation)/len(wsn)})
        
gurobi_sf_data = pd.DataFrame(gurobi_sf_data)
to_cache('gurobi_sf', gurobi_sf_data)
len(gurobi_sf_data)

  0%|          | 0/1 [00:00<?, ?it/s]

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
[client] info  : Validate web license token 813d29df-e6ac-42e9-bff6-2178b73d0b3f for container d948a34dccd1eeb65e36687706e634ec1f7776fa67e1ab70215ca0163d01bee1
[client] error : Web license token expired
[client] info  : Validate web license token bc1d5a5e-c469-4e39-b2c9-57ce52328337 for container d948a34dccd1eeb65e36687706e634ec1f7776fa67e1ab70215ca0163d01bee1
Academic license - for non-commercial use only - registered to josefrf@ufmg.br
[client] info  : Web license token valid


1

In [5]:
eval_data, cache = get_cache('eval')
for ser_instance in tqdm(list_serialized_instances):
    if ser_instance in cache: continue   
    result_evaluator = subprocess.run([
        "time", "-f", "%P %S %U",
        "/builds/instance_evaluator", 
        str(K), str(M), ser_instance
    ], capture_output=True)
    
    assert int(result_evaluator.returncode) == 0
    
    output = result_evaluator.stdout.decode()
    i, j = output.replace('\t', '').split('|')
    status = (i.split(':')[-1].strip(), j.split(':')[-1].strip())
    feasible = status == ('SUCCESS', 'SUCCESS')
    
    cpu, systime, usertime = map(
        float,
        result_evaluator.stderr.decode().replace('%', 'e-2').split(' ')
    )
    
    eval_data.append({
        'serialized_instance': ser_instance,
        'eval_status': status, 'eval_feasible': feasible,
        'eval_runtime': (systime+usertime)*cpu,
        'eval_cpu': cpu, 'eval_sys_time': systime, 'eval_user_time': usertime
    })
    
eval_data = pd.DataFrame(eval_data)
to_cache('eval', eval_data)
len(eval_data)

  0%|          | 0/1 [00:00<?, ?it/s]

1011

In [6]:
df = (eval_data
      .merge(gurobi_mf_data)
      .merge(gurobi_sf_data)
).sort_values('serialized_instance').reset_index(drop=True).copy()

In [7]:
len(df)

1

In [8]:
df.loc[:, 'obj_instance'] = df['serialized_instance'].apply(lambda i: KCMC_Instance(i, False, True, True))

In [9]:
df.loc[:, 'instance_size'] = df['obj_instance'].apply(lambda i: i.num_sensors)

In [10]:
df['instance_size'].unique()

array([100])

In [12]:
df

,serialized_instance,eval_status,eval_feasible,eval_runtime,eval_cpu,eval_sys_time,eval_user_time,gurobi_mf_status,gurobi_mf_gap,gurobi_mf_feasible,gurobi_mf_runtime,gurobi_sf_gap,gurobi_sf_status,gurobi_sf_feasible,gurobi_sf_runtime,gurobi_sf_installation,gurobi_sf_size,gurobi_sf_quality,obj_instance,instance_size
0,KCMC;100 100 1;300 50 100;170369128;PS;0 6;0 8...,"(SUCCESS, SUCCESS)",True,0.0282,0.94,0.0,0.03,LIMIT,inf,False,60.092767,0.0,OPTIMAL,True,41.051327,"{'i6': '0', 'i7': '0', 'i8': '0', 'i14': '0', ...",18,0.191489,<KCMC_0.1:100:100:1:300:50:100 170369128 [0]>,100


In [11]:
df[['instance_size', 'gurobi_mf_runtime']]

fig = go.Figure()

fig.add_trace(go.Bar(x=df['instance_size'], y=df['gurobi_mf_runtime'],
                         name='GUROBY MULTI'))

fig.show()

NameError: name 'go' is not defined

In [ ]:
df_diff = df[
  ((
       (df['gurobi_mf_feasible'] != df['eval_feasible'])
     | (df['gurobi_sf_feasible'] != df['eval_feasible'])
     | (df['gurobi_mf_feasible'] != df['gurobi_sf_feasible'])
     | (df['gurobi_mf_size'].fillna(-1) != df['gurobi_sf_size'].fillna(-1))
   ) & (df['gurobi_mf_status'] != 'LIMIT')
     & (df['gurobi_sf_status'] != 'LIMIT'))
]

assert len(df_diff) == 0

In [ ]:
VIEW = [#'serialized_instance',
    'gurobi_mf_feasible', 'gurobi_sf_feasible', 'eval_feasible',
    # 'gurobi_mf_status', 'gurobi_sf_status', 'eval_status',
    'gurobi_mf_size', 'gurobi_sf_size',
    'gurobi_mf_runtime', 'gurobi_sf_runtime', 'eval_runtime',
    # 'eval_sys_time', 'eval_user_time',  'eval_cpu'
]



df_diff[VIEW]

In [ ]:
!pip install plotly

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(x=df.index, y=df['gurobi_mf_runtime'],
                         name='GUROBY MULTI'))
# fig.add_trace(go.Scatter(x=df.index, y=df['gurobi_sf_runtime'],
#                          mode='lines', name='GUROBY SINGLE'))
# fig.add_trace(go.Scatter(x=df.index, y=df['eval_runtime']*100,
#                          mode='lines', name='EVAL x100'))

fig.show()

In [ ]:
df[VIEW].sort_values('gurobi_sf_runtime')

In [ ]:
ROW = df.loc[55]
KCMC_Instance(ROW['serialized_instance'], False, True, True).plot(
    labels=True, installation=ROW['gurobi_mf_installation'], minimal=True)

In [ ]:
KCMC_Instance(ROW['serialized_instance'], False, True, True).plot(
    labels=True, installation=ROW['gurobi_sf_installation'], minimal=True)

In [ ]:
ROW['gurobi_sf_installation']